## Budget for Happiness
### Mike Winton

Write a function to provide budgeting recommendations to maximize happiness via discretionary spending.

Inputs:
- List of expected yearly incomes
- List of expected yearly mandatory expenses

Output:
- List of recommended discretionary expenditures

Notes:
- Can't assume each year's income covers that year's expenses; may need to roll funds forward.
- The person definitely shouldn’t die (i.e. budget must cover mandatory expenses every year).
- Happiness comes from discretionary spending.
- Happiness vs. Discretionary expenditures plot starts steep and flattens out.  In other words, the marginal value of each dollar declines over the previous one throughout the year (i.e. 1st dollar generates more happiness than the second.  Second generates more than the third, etc...).
- Marginal value of each discretionary dollar resets each year.
- Use up all money by end of life.

In [1]:
def make_me_happy(income, mandatory_exp):
    """
    Calculates how to spread discretionary income throughout lifetime for max happiness.
    
    Inputs:
        income = list(float) of expected future income per year
        mandatory_exp = list(float) of expected mandatory expenses per year
        
    Returns:
        discretionary = list(float) of discretionary expenditures per year to maximize happiness
    """
    
    # pad with zeroes if lists aren't the same length
    if len(income) > len(mandatory_exp):
        mandatory_exp += [0] * (len(income) - len(mandatory_exp))
    elif len(income) < len(mandatory_exp):
        income += [0] * (len(mandatory_exp) - len(income))
    
    # fail if total income is insufficient for total expenses
    if sum(mandatory_exp) > sum(income):
        raise ValueError('Impossible!')
        
    num_years = len(income)  # for convenience later
    
    # initialize variables
    first_surplus_idx = -1  # -1 means it hasn't been found yet
    rollover_needed = 0
    mandatory_exp_and_reserves = [0] * num_years
    discretionary = [0] * num_years 
    
    # iterate *backwards* from last year of life
    for idx in range(num_years-1, -1, -1):
        # Any future shortage that was passed back gets added to the
        # current year's mandatory expenses
        if rollover_needed > 0:
            mandatory_exp_and_reserves[idx] = mandatory_exp[idx] + rollover_needed
        else:
            mandatory_exp_and_reserves[idx] = mandatory_exp[idx]
        
        # Check see if there's a surplus after accounting for future reserves
        surplus = income[idx] - mandatory_exp_and_reserves[idx]  
        if surplus < 0:
            # Pass shortfall back to prior year so it can be budgeted for
            rollover_needed = -surplus
        else:
            # Distribute any surplus
            if first_surplus_idx == -1:
                # The first time we see a surplus is easiest (i.e. the latest year);
                # we just distribute evently to the current and all future years, 
                # without any additional checks on the amount.
                num_years_to_distribute = num_years - idx
                yearly_distribution = surplus / num_years_to_distribute
                # iterate from current through final year, assign discretionary funds
                for redist_idx in range(idx, num_years):
                    discretionary[redist_idx] = yearly_distribution
            else:
                # This branch means it's not the first time a surplus was seen; we
                # determine whether to also spread across any previously calculated years
                # or whether to just distribute to years earlier than the previously
                # set surplus.  We don't want to blindly distribute through to the end
                # of life, because if there are already more significant discretionary
                # expenditures planned, the marginal value of increases those expenses
                # is worth less than to leave it in lower-spending years.
                #
                # First, calculate as if we would only distribute up until the point of 
                # the previously identified surplus.
                num_years_to_distribute = first_surplus_idx - idx
                yearly_distribution = surplus / num_years_to_distribute
                
                # If existing (later) distribution is a higher $ amount, then only
                # distribute this money in years earlier than that point.
                if yearly_distribution <= discretionary[first_surplus_idx]:
                    for redist_idx in range(idx, first_surplus_idx):
                        discretionary[redist_idx] = yearly_distribution
                else:
                    # This branch means the surplus is high enough that we should
                    # spread over all remaining years, so first we need to pool with
                    # the previously distributed discretionary funds in later years.
                    for redist_idx in range(first_surplus_idx, num_years):
                        surplus += discretionary[redist_idx]
                    num_years_to_distribute = num_years - idx
                    yearly_distribution = surplus / num_years_to_distribute
                    for redist_idx in range(idx, num_years):
                        discretionary[redist_idx] = yearly_distribution
            # Reset pointer to the first year with a surplus
            first_surplus_idx = idx

    print(f'{"Year": >6}{"Income": >10}{"Mandatory Exp": >16}{"Mand. Exp+Reserves": >20}{"Discretionary": >16}')
    for idx in range(0, num_years):
        print(f'{idx: >6}{income[idx]: >10}{mandatory_exp[idx]: >16}'
              f'{mandatory_exp_and_reserves[idx]: >20}{discretionary[idx]: >16}')
        
    print(f'\nTotal income = {sum(income)}, total mandatory expenses = '
          f'{sum(mandatory_exp)}, total discretionary = {sum(discretionary)}')
        
    return discretionary

In [2]:
# Straightforward example (increasing income)
income = [1000, 1200, 1400, 1600]
mandatory_exp = [1000, 1000, 1000, 1000]
discretionary = make_me_happy(income, mandatory_exp)

  Year    Income   Mandatory Exp  Mand. Exp+Reserves   Discretionary
     0      1000            1000                1000             0.0
     1      1200            1000                1000           200.0
     2      1400            1000                1000           400.0
     3      1600            1000                1000           600.0

Total income = 5200, total mandatory expenses = 4000, total discretionary = 1200.0


In [3]:
# Straightforward example (decreasing income)
income = [1600, 1400, 1200, 1000]
mandatory_exp = [1000, 1000, 1000, 1000]
discretionary = make_me_happy(income, mandatory_exp)

  Year    Income   Mandatory Exp  Mand. Exp+Reserves   Discretionary
     0      1600            1000                1000           300.0
     1      1400            1000                1000           300.0
     2      1200            1000                1000           300.0
     3      1000            1000                1000           300.0

Total income = 5200, total mandatory expenses = 4000, total discretionary = 1200.0


In [4]:
# More complicated example (income is sometimes below expenses)
income = [1400, 1000, 800, 1000, 825, 815]
mandatory_exp = [1000, 1000, 900, 1100, 800, 800]
discretionary = make_me_happy(income, mandatory_exp)

  Year    Income   Mandatory Exp  Mand. Exp+Reserves   Discretionary
     0      1400            1000                1200            40.0
     1      1000            1000                1200            40.0
     2       800             900                1000            40.0
     3      1000            1100                1100            40.0
     4       825             800                 800            40.0
     5       815             800                 800            40.0

Total income = 5840, total mandatory expenses = 5600, total discretionary = 240.0


In [5]:
# More complicated example (income is sometimes below expenses)
income = [1000, 900, 800, 0, 500]
mandatory_exp = [500, 700, 300, 400, 900]
discretionary = make_me_happy(income, mandatory_exp)

  Year    Income   Mandatory Exp  Mand. Exp+Reserves   Discretionary
     0      1000             500                 600            80.0
     1       900             700                1000            80.0
     2       800             300                1100            80.0
     3         0             400                 800            80.0
     4       500             900                 900            80.0

Total income = 3200, total mandatory expenses = 2800, total discretionary = 400.0


In [6]:
# More complicated example (can't redistribute earlier surplus to all future years)
income = [1400, 1000, 800, 1000, 1000]
mandatory_exp = [1000, 1000, 900, 1100, 800]
discretionary = make_me_happy(income, mandatory_exp)

  Year    Income   Mandatory Exp  Mand. Exp+Reserves   Discretionary
     0      1400            1000                1200            50.0
     1      1000            1000                1200            50.0
     2       800             900                1000            50.0
     3      1000            1100                1100            50.0
     4      1000             800                 800           200.0

Total income = 5200, total mandatory expenses = 4800, total discretionary = 400.0


In [7]:
# More complicated example (can't redistribute earlier surplus to all future years)
income = [1212, 1000, 800, 1000, 825, 815]
mandatory_exp = [1000, 1000, 900, 1100, 800, 800]
discretionary = make_me_happy(income, mandatory_exp)

  Year    Income   Mandatory Exp  Mand. Exp+Reserves   Discretionary
     0      1212            1000                1200             3.0
     1      1000            1000                1200             3.0
     2       800             900                1000             3.0
     3      1000            1100                1100             3.0
     4       825             800                 800            20.0
     5       815             800                 800            20.0

Total income = 5652, total mandatory expenses = 5600, total discretionary = 52.0
